In [1]:
import keras
#import keras_resnet.models
from keras.models import Model
from keras.layers import Input, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, AveragePooling2D, concatenate, \
    Activation, ZeroPadding2D
from keras.layers import add, Flatten, GlobalAveragePooling2D
from keras.utils import plot_model
from keras.metrics import top_k_categorical_accuracy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import ReduceLROnPlateau, CSVLogger,ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras import backend as K
from keras import optimizers
from keras import applications
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
#from keras.applications.vgg16 import VGG16
#from keras.applications.vgg16 import preprocess_input
#from keras.applications import vgg16
from sklearn.utils import shuffle


Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
import numpy as np
import glob
import cv2
import os
data_src = '//data/examples/flower_classification/'

In [4]:
'''設定參數'''
classn = 5
batch_size = classn * 2
epochs = 80
lr = 0.0001
train_ratio= 0.8
sz=299
r_state = 40
channel = 3

In [5]:
labelmap=pd.read_csv(data_src+'mapping.csv',names=['dirs','class'])

In [6]:
sortlabel=labelmap.sort('class')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  """Entry point for launching an IPython kernel.


In [7]:
targetlist=sortlabel.dirs.as_matrix().tolist()
targetlist[:5]

['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

targetlist

In [8]:
x=[]
y=[]
for i,c in enumerate(targetlist):
    ff=glob.glob((data_src+'train/'+c+'/*jpg'))
    for f in ff:
        img=cv2.imread(f )
        img=cv2.resize(img,(sz,sz))
        img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img[:,:,::-1] # cv2 預設讀進來是 BGR, 我們要轉回 RGB
        # zero-mean
        # pre-trained model 使用 ImageNet 做訓練
        # ImageNet 的所有影像 RGB 平均值 [123.68, 116.78, 103.94]
        #img = img.astype('float32') - np.array([123.68, 116.78, 103.94])
        img = img.astype('float32')
        x.append(img)
        nn=np.zeros(classn,dtype=float)
        nn[i]=1
        y.append(nn)
xx=np.array(x)
yy=np.array(y)

xx = preprocess_input(xx)

In [9]:
xx.shape

(3823, 299, 299, 3)

In [10]:
yy.shape

(3823, 15)

In [11]:
#----------------------------資料training set, testing set 分割---------------------------------------

x_train, x_valid, y_train, y_valid = train_test_split(xx, 
                                                      yy, 
                                                      test_size = 1.0 - train_ratio,
                                                      stratify = y,
                                                      random_state = r_state)
#------------------------------------檢查資料---------------------------------------------------------
print("training set data dimension")
print(x_train.shape)
print(y_train.shape)
print("-----------")
print("training set: %i" % len(x_train))
print("validation set: %i" % len(x_valid))

training set data dimension
(3058, 299, 299, 3)
(3058, 15)
-----------
training set: 3058
validation set: 765


x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')


# subtract mean and normalize
mean_image = np.mean(x_train, axis=0)
x_train -= mean_image
x_valid -= mean_image
x_train /= 255.
x_valid /= 255.

In [12]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.01), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.01, patience=10)
mcp_save = ModelCheckpoint('xception-best.hdf5', save_best_only=True, monitor='val_loss', mode='min')
csv_logger = CSVLogger('resnet18.csv')

In [13]:
callbacks=[lr_reducer, early_stopper, csv_logger,mcp_save]

In [14]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=True,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        shear_range = 0.1,
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
# datagen.fit(x_train)

In [15]:
from keras.applications.densenet import DenseNet201
model = DenseNet201(weights = "imagenet", include_top=False, input_shape = (sz, sz, channel))

# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
#for layer in model.layers:
#    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
#x = GlobalAveragePooling2D(data_format= "channels_last")(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(15, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer=optimizers.Adam(lr=lr), metrics=["accuracy"])

#model_final.summary()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [16]:
history = model_final.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                                    steps_per_epoch=x_train.shape[0] / batch_size,
                                    validation_data=(x_valid, y_valid),  
                                    epochs=epochs, 
                                    callbacks=[lr_reducer, early_stopper, csv_logger,mcp_save])

Epoch 1/80
306/305 [==============================] - 327s 1s/step - loss: 1.7288 - acc: 0.7260 - val_loss: 0.5978 - val_acc: 0.8797
Epoch 2/80
306/305 [==============================] - 155s 506ms/step - loss: 0.7930 - acc: 0.8289 - val_loss: 0.5870 - val_acc: 0.8784
Epoch 3/80
306/305 [==============================] - 155s 507ms/step - loss: 0.4944 - acc: 0.8727 - val_loss: 0.2949 - val_acc: 0.9046
Epoch 4/80
306/305 [==============================] - 160s 522ms/step - loss: 0.4094 - acc: 0.8915 - val_loss: 0.2921 - val_acc: 0.9111
Epoch 5/80
306/305 [==============================] - 119s 389ms/step - loss: 0.3581 - acc: 0.8991 - val_loss: 0.4191 - val_acc: 0.8967
Epoch 6/80
306/305 [==============================] - 119s 388ms/step - loss: 0.4248 - acc: 0.8908 - val_loss: 0.3859 - val_acc: 0.9111
Epoch 7/80
306/305 [==============================] - 119s 387ms/step - loss: 0.5555 - acc: 0.8724 - val_loss: 0.9868 - val_acc: 0.7922
Epoch 8/80
306/305 [==============================]

In [17]:
testdata=pd.read_csv(data_src+'submission.csv')

testdata

In [18]:
testdataindexlist=testdata.id.as_matrix().tolist()

In [19]:
x_t=[]
for f in testdataindexlist:
    img=cv2.imread(data_src+'test/'+f+'.jpg')
    img=cv2.resize(img,(sz,sz))
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img[:,:,::-1] # cv2 預設讀進來是 BGR, 我們要轉回 RGB
    #img = img.astype('float32') - np.array([123.68, 116.78, 103.94])
    img = img.astype('float32')
    x_t.append(img)

        
x_test=np.array(x_t)
x_test = preprocess_input(x_test)

In [20]:
#%%script false
model_final.load_weights(filepath = 'xception-best.hdf5')

score = model_final.evaluate(x_train, y_train, verbose=2)
print("Best model")
print('\t-CV loss:', score[0])
print('\t-CV accuracy:', score[1])

pt = model_final.predict(x_train)
mse = (np.mean((pt-y_train)**2))
print('\t-CV MSE: ', mse)


Best model
	-CV loss: 0.0326318996523
	-CV accuracy: 0.989535644212
	-CV MSE:  0.00100727563216


In [21]:
predicts = model_final.predict(x_test, verbose=1)

500/500 [==============================] - 5s 10ms/step


In [22]:
result = []
codeMap = {}
maxPossible = np.argmax(predicts,axis=1)
for i in range(0, len(predicts)):
    filename = testdata.id[i]
    code = maxPossible[i]
    codeMap[code] = code
#     print(i)
    result.append([filename,code])
result[:5]

[['100080576_f52e8ee070_n', 0],
 ['1008566138_6927679c8a', 3],
 ['100930342_92e8746431_n', 4],
 ['1022552002_2b93faf9e7_n', 3],
 ['1022552036_67d33d5bd8_n', 3]]

In [23]:
answer=pd.DataFrame(result,columns=['id','class'])

In [24]:
answer.to_csv('xception-submission.csv', index = False)